In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/OHrydko/Machine-Learning-Labs/master/lab1/MP-07-Hrydko.csv', 
                 header= None, names=['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'y'], sep=';',engine='python', nrows=5)
df.head()

,x0,x1,x2,x3,x4,x5,y
0,6248.00,131.3700,260.7370,5229.49,157,25.8162,9.726
1,2114.97,56.1939,97.7716,3601.37,27,39.8544,17.395
2,1164.74,45.0019,63.9749,2026.36,22,42.8624,2.375
3,2192.61,55.3350,93.4638,3336.69,25,38.2677,12.987
4,1168.13,53.4750,63.4258,2812.04,17,39.8232,5.562


In [8]:
train = df.iloc[:200]
# визначення параметрів
system_len = len(train.columns)
alpha = 0.9
# знаходження коефіцієнтів
print(train[train.columns[:-1]].values)
matrix_x = train[train.columns[:-1]].values
solve = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(matrix_x), matrix_x)), np.transpose(matrix_x)), train['y'].values)

# Оцінка статистичної значимості коефіцієнтів
# Відхилення
q_e = (train['y'] - (np.dot(train[train.columns[:-1]],solve)))
# Оцінка дисперсії
s_2 = (q_e**2).sum()/(len(train) - system_len)
# Коефіцієнти зв'язку
K_coef = np.sqrt(np.diag(s_2 * np.linalg.inv(np.dot(np.transpose(matrix_x), matrix_x))))

t_student = round(sts.t.ppf((1 + alpha)/2, len(train) - system_len), 2)
t_calculated = (np.absolute(solve)/K_coef).round(2)
weight_coef = t_calculated > t_student

# Розрахунок коефіцієнта множинної детермінації
R = round(np.sqrt((np.dot(np.dot(np.transpose(solve), np.transpose(matrix_x)), train['y'].values) -
             len(train) * train['y'].mean())/\
    (np.dot(np.transpose(train['y'].values), train['y'].values) - len(train) * train['y'].mean())), 2)
print('Значення розподілу t = {} при параметрах alpha = {} & p = {} при'.format(t_student, alpha, len(train) - system_len))
for i in range(len(solve)):
    print('b{} = {} значущий {} значення t роподілу = {} '
          .format(i, round(solve[i], 2), weight_coef[i], t_calculated[i]))
print('Коефіцієнт детермінації R = {}, R^2 = {},оцінка дисперції s^2 = {}'.format(R, round(R**2, 2), round(s_2, 2)))

[[6248.      131.37    260.737  5229.49    157.       25.8162]
 [2114.97     56.1939   97.7716 3601.37     27.       39.8544]
 [1164.74     45.0019   63.9749 2026.36     22.       42.8624]
 [2192.61     55.335    93.4638 3336.69     25.       38.2677]
 [1168.13     53.475    63.4258 2812.04     17.       39.8232]]
Значення розподілу t = nan при параметрах alpha = 0.9 & p = -2 при
b0 = -0.0 значущий False значення t роподілу = nan 
b1 = -0.85 значущий False значення t роподілу = nan 
b2 = 0.32 значущий False значення t роподілу = nan 
b3 = 0.01 значущий False значення t роподілу = nan 
b4 = -0.03 значущий False значення t роподілу = nan 
b5 = -0.06 значущий False значення t роподілу = nan 
Коефіцієнт детермінації R = 0.91, R^2 = 0.83,оцінка дисперції s^2 = -20.13


<ipython-input-8-f6e32df533ae>:16: RuntimeWarning: invalid value encountered in sqrt
  K_coef = np.sqrt(np.diag(s_2 * np.linalg.inv(np.dot(np.transpose(matrix_x), matrix_x))))
<ipython-input-8-f6e32df533ae>:20: RuntimeWarning: invalid value encountered in greater
  weight_coef = t_calculated > t_student


In [61]:
def q_func(w0, wi):
    return (((w0 + train[train.columns[:-1]] * wi).sum(axis = 1) - train['y'])**2).mean()

# Градієнт функції
def gradient(w0, wi):
    w_0 = (2/len(matrix_x)) * ((w0 + train[train.columns[:-1]] * wi).sum(axis = 1) - train['y']).sum()
    w_i = list((2/len(matrix_x)) * \
    np.dot(np.transpose(matrix_x), ((w0 + train[train.columns[:-1]] * wi).sum(axis = 1) - train['y'])))
    w_i.append(w_0)
    return w_i

def answer(w_arg):
    lam = 0.6
    return w_arg - lam * np.array(gradient(w_arg[-1], w_arg[:-1])) / np.sqrt((np.array(gradient(w_arg[-1], w_arg[:-1]))**2).sum())

In [62]:
w = [np.random.uniform(0, 1, size=system_len)]
epsilon = 0.001
error = []
k = [0]
while True:
    w.append(answer(w[-1]))
    error.append(np.absolute(q_func(w[-1][-1], w[-1][:-1]) - q_func(w[-2][-1], w[-2][:-1])))
    print('Ітерація № = {} Значення помилки = {} \n Значення аргументів {}'.format(k[-1], error[-1], w[-1]))
    if error[-1] < epsilon:
        break
    k.append(k[-1] + 1) 
coef_regretion = w[-1]
print('Ітерацій = {} Помилка = {} \n Вектор значень аргументів {}'.format(k[-1], error[-1], coef_regretion))

Ітерація № = 0 Значення помилки = 6.917653053682372 
 Значення аргументів [0.60920223 1.09972342 1.0984256  0.46125535 0.87017579 0.65926162
 0.41351734 0.39796971 0.68433031]
Ітерація № = 1 Значення помилки = 5.183148890150562 
 Значення аргументів [0.39078021 0.79827184 1.00181997 0.23465665 0.70914713 0.55121831
 0.34538404 0.23177601 0.38287872]
Ітерація № = 2 Значення помилки = 4.846261113812382 
 Значення аргументів [0.49600262 1.13678557 1.13161367 0.310148   0.81868408 0.63572055
 0.43128378 0.49763698 0.72139246]
Ітерація № = 3 Значення помилки = 3.839433402237397 
 Значення аргументів [0.30087288 0.82219453 1.02863572 0.11500217 0.66863592 0.53204259
 0.36031182 0.30527692 0.40680142]
Ітерація № = 4 Значення помилки = 3.5717530362565952 
 Значення аргументів [0.4044562  1.16065553 1.15968102 0.18816748 0.77778869 0.61626132
 0.44714571 0.57189558 0.74526242]
Ітерація № = 5 Значення помилки = 2.816192921823074 
 Значення аргументів [0.2189728  0.84147098 1.05467863 0.00579056 

Ітерація № = 47 Значення помилки = 4.054682677801523 
 Значення аргументів [-1.14159919  0.99469777  1.82136354 -1.94720567  0.15395699  0.24071998
  0.8963218   1.26234395  0.57930465]
Ітерація № = 48 Значення помилки = 3.996760510619872 
 Значення аргументів [-1.04458211  1.3283353   1.97551779 -1.89356611  0.26696389  0.32458304
  1.00059458  1.52813065  0.91294218]
Ітерація № = 49 Значення помилки = 4.068883867094492 
 Значення аргументів [-1.19583456  0.99607182  1.86876995 -2.03524942  0.13958957  0.23030849
  0.92968372  1.29421797  0.5806787 ]
Ітерація № = 50 Значення помилки = 4.002382649470874 
 Значення аргументів [-1.09795289  1.32952013  2.02330321 -1.9810631   0.25306755  0.31434778
  1.034219    1.55947155  0.91412702]
Ітерація № = 51 Значення помилки = 4.068543424983673 
 Значення аргументів [-1.24879866  0.99712553  1.91670113 -2.12248197  0.1258999   0.22014595
  0.96340711  1.3252708   0.58173241]
Ітерація № = 52 Значення помилки = 3.994387153383073 
 Значення аргуме

  1.74696963  2.06567887  0.89530032]
Ітерація № = 93 Значення помилки = 2.3764644767032053 
 Значення аргументів [-2.08285597  0.97672023  2.95051334 -3.74424542 -0.04663479  0.05356484
  1.67800146  1.82875892  0.56132712]
Ітерація № = 94 Значення помилки = 2.262331896922518 
 Значення аргументів [-1.96616477  1.30869929  3.10182393 -3.67189555  0.07360103  0.14082961
  1.77939034  2.08465358  0.89330618]
Ітерація № = 95 Значення помилки = 2.26138470927868 
 Значення аргументів [-2.1116839   0.97468199  2.99880873 -3.81274414 -0.051214    0.04721477
  1.71050266  1.84769247  0.55928888]
Ітерація № = 96 Значення помилки = 2.147838400115951 
 Значення аргументів [-1.99439529  1.30665726  3.14979567 -3.73969095  0.06917205  0.13456965
  1.81161872  2.10332161  0.89126415]
Ітерація № = 97 Значення помилки = 2.145309501535067 
 Значення аргументів [-2.13976318  0.97259742  3.04695602 -3.88064731 -0.05558995  0.04095544
  1.74281143  1.86632389  0.55720431]
Ітерація № = 98 Значення помилки

Ітерація № = 141 Значення помилки = 0.4887360410286874 
 Значення аргументів [-2.61293085  0.91989326  4.07812807 -5.27079874 -0.11696199 -0.08229265
  2.4111667   2.22424676  0.50450015]
Ітерація № = 142 Значення помилки = 0.5813113669497767 
 Значення аргументів [-2.48668306  1.25207825  4.222552   -5.18597333  0.00530146  0.00635266
  2.50685688  2.47594336  0.83668514]
Ітерація № = 143 Значення помилки = 0.6075086555952467 
 Значення аргументів [-2.6294213   0.91732332  4.12431597 -5.331078   -0.11866785 -0.08752802
  2.44001151  2.23898743  0.50193021]
Ітерація № = 144 Значення помилки = 0.6991230504657011 
 Значення аргументів [-2.50292906e+00  1.24952236e+00  4.26850557e+00 -5.24588657e+00
  3.63725601e-03  1.15137525e-03  2.53551062e+00  2.49057527e+00
  8.34129249e-01]
Ітерація № = 145 Значення помилки = 0.7259514225345711 
 Значення аргументів [-2.64557686  0.91474554  4.1705026  -5.39123715 -0.12030794 -0.09275071
  2.46876034  2.25365002  0.49935242]
Ітерація № = 146 Значен

Ітерація № = 189 Значення помилки = 3.2085775303052912 
 Значення аргументів [-2.9316054   0.85696305  5.20183927 -6.71079376 -0.14298501 -0.20780046
  3.08600259  2.56565261  0.44156994]
Ітерація № = 190 Значення помилки = 3.2800386193422497 
 Значення аргументів [-2.80120438  1.18948056  5.34165591 -6.61913926 -0.0200689  -0.11857564
  3.17795312  2.81542492  0.77408745]
Ітерація № = 191 Значення помилки = 3.3139880228699425 
 Значення аргументів [-2.94198507  0.85432195  5.2501209  -6.77168951 -0.14349738 -0.21317133
  3.1137515   2.57970647  0.43892883]
Ітерація № = 192 Значення помилки = 3.384663409582899 
 Значення аргументів [-2.81146824  1.18685238  5.38978524 -6.67981962 -0.02056418 -0.12393025
  3.20557853  2.82942116  0.77145927]
Ітерація № = 193 Значення помилки = 3.418573813542146 
 Значення аргументів [-2.95217152  0.85168229  5.29858595 -6.83275244 -0.14396874 -0.21856512
  3.14150434  2.59377544  0.43628918]
Ітерація № = 194 Значення помилки = 3.488470904755964 
 Значен

KeyboardInterrupt: 

In [68]:
test = df.iloc[200:]
error_2_mnk = (test['y'] - (np.dot(test[test.columns[:-1]],solve)))
error_2_grad = (test['y'] - (coef_regretion[-1] + test[test.columns[:-1]] * coef_regretion[:-1]).sum(axis = 1))
print('Похибка МНК = {} \nПохибка градієнта = {}'.format(error_2_mnk, error_2_grad))
    
error_2_mnk = ((test['y'] - (np.dot(test[test.columns[:-1]],solve)))**2).mean()
error_2_grad = ((test['y'] - (coef_regretion[-1] + test[test.columns[:-1]] * coef_regretion[:-1]).sum(axis = 1))**2).mean()
print('Похибка МНК = {} \nПохибка градієнта = {}'.format(error_2_mnk, error_2_grad))

Похибка МНК = 200    -3.678071
201     0.917266
202     5.708631
203     8.074346
204    -6.592984
205    -1.151232
206     4.122168
207    -0.780396
208    13.937866
209     4.828391
210    13.565890
211     4.028181
212    -5.944858
213    -2.403940
214     5.271251
215     7.645430
216    -3.246371
217    -3.890983
218    -5.092791
219     5.424096
220    -3.183289
221     4.874486
222    -2.803816
223     6.629483
224     3.738635
225     3.217957
226     6.438337
227    -0.922296
228    -1.638576
229    -2.337757
230     4.646462
231    11.945684
232     1.268816
233    -4.134670
234     3.213958
235     6.681204
236     7.674559
237    -3.198521
238     5.729535
239     9.157382
240     6.708698
241    -1.291571
242     1.727304
243    -1.914989
244    -3.303058
245    10.786561
246     1.538467
247     4.903752
248     0.900929
249     7.266463
Name: y, dtype: float64 
Похибка градієнта = 200    -5.448328
201    -0.741160
202     3.910047
203     6.212012
204    -8.567569
205   